In [126]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim  as optim
import numpy as np
import matplotlib.pyplot as plt
import time
from copy import deepcopy

device = "cuda" if torch.cuda.is_available() else "cpu"

In [127]:
data = torch.randn((100, 30, 170), requires_grad = True, device = device)
target_ = torch.randint(0, 2, (100,), device = device)

trainloader = DataLoader(TensorDataset(data, target_), 
                         batch_size=50, 
                         shuffle=True)

In [128]:
class LSTMEEG(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(LSTMEEG, self).__init__()
        
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, batch_first = True)
        self.fc = nn.Linear(hidden_dim, 1)
        self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax()
    

    def init_hidden(self, batch):
        '''Initialize hidden states for LSTM cell.'''
        device = self.lstm.weight_ih_l0.device
        return (torch.zeros(1, batch, self.hidden_dim, device=device),
                torch.zeros(1, batch, self.hidden_dim, device=device))
    
    def forward(self, X, hidden = None):
        if hidden is None:
            hidden = self.init_hidden(X.shape[0])
            
        lstm_out, (h, c) = self.lstm(X, hidden)
        y = self.fc(lstm_out[:, -1])
        y = self.sigmoid(y) # binary classification
        # y = self.softmax(y) # multi-class
        
        return y

In [129]:
def restore_parameters(model, best_model):
    '''Move parameter values from best_model to model.'''
    for params, best_params in zip(model.parameters(), best_model.parameters()):
        params.data = best_params

def train_LSTMEEG(model, trainloader, epochs, lr, device):
    model.to(device)
    loss_fn = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    history = {'loss' : []}
    
    best_it = None
    best_loss = np.inf
    best_model = None
    
    for epoch in range(epochs):
        losses = []
        for X, Y in trainloader:
            model.zero_grad()
            pred = model(X).squeeze(1)
            loss = loss_fn(pred, Y.float())
            
            loss.backward()
            optimizer.step()
            losses.append(float(loss))
        avg_loss = np.mean(losses)
        history['loss'].append(avg_loss)
        print("Epoch {} / {}: Loss = {:.3f}".format(epoch+1, epochs, avg_loss))
        
        if best_loss > avg_loss:
            best_loss = avg_loss
            best_model =  deepcopy(model)
            
    restore_parameters(model, best_model)
    
    return history

In [131]:
model = LSTMEEG(170, 50)
his = train_LSTMEEG(model, trainloader, 10, 0.01, device= device)

Epoch 1 / 10: Loss = 0.681
Epoch 2 / 10: Loss = 0.532
Epoch 3 / 10: Loss = 0.396
Epoch 4 / 10: Loss = 0.264
Epoch 5 / 10: Loss = 0.155
Epoch 6 / 10: Loss = 0.081
Epoch 7 / 10: Loss = 0.040
Epoch 8 / 10: Loss = 0.019
Epoch 9 / 10: Loss = 0.010
Epoch 10 / 10: Loss = 0.006
